In [17]:
! pwd
DATASET = 'ImageNet'
data_path = '/nobackup-slow/dataset/ILSVRC-2012/train' if DATASET == 'ImageNet' else '/nobackup-slow/dataset/ImageNet_OOD_dataset/{}'.format(DATASET)

/afs/cs.wisc.edu/u/z/i/ziyangc/Github/CLIP_experiment/image_captioning


In [18]:
import sys
sys.path.append('/nobackup-fast/ziyangc/scene_graph_benchmark')
import maskrcnn_benchmark

In [20]:
import base64
import cv2
import json
import os.path as op
import os
import yaml
from tqdm import tqdm
from maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer
from maskrcnn_benchmark.structures.tsv_file_ops import generate_linelist_file
from maskrcnn_benchmark.structures.tsv_file_ops import generate_hw_file
from maskrcnn_benchmark.structures.tsv_file import TSVFile
from maskrcnn_benchmark.data.datasets.utils.image_ops import img_from_base64

# To generate a tsv file:
get_class_name = {
    'ImageNet': lambda p, f: p.split('/')[-1],
    'iNaturalist': lambda p, f: 'OOD',
    'Places': lambda p, f: f.split('_')[1],
    'Textures': lambda p, f: p.split('/')[-1]
}

filter_classes = {
    'ImageNet': True,
    'iNaturalist': False,
    'Places': False,
    'Textures': False
}

test_classes = {'n04552348', 'n04285008', 'n01530575', 'n02123597', 'n02422699', 'n02107574', 'n01641577', 'n01728572', 'n03095699', 'n03417042'}
tsv_data_path = 'scene_graph_benchmark/tools/mini_tsv/data/'
ImageNet_classes = {key: string for [key, string] in json.load(open("imagenet_class_index.json")).values()}
img_list = [(path, f) for (path, _, files) in os.walk(data_path) for f in files if (f.lower().endswith(".jpeg") or
f.lower().endswith('jpg')) and (not filter_classes[DATASET] or path.split('/')[-1] in test_classes)]
tsv_file = "{}.test.tsv".format(DATASET)
label_file = "{}.test.label.tsv".format(DATASET)
hw_file = "{}.test.hw.tsv".format(DATASET)
linelist_file = "{}.test.linelist.tsv".format(DATASET)

rows = []
rows_label = []
rows_hw = []
for (path, filename) in tqdm(img_list):
    class_name =  get_class_name[DATASET](path, filename)
    img_key = class_name + '_' + filename.split('.')[0]
    img_path = op.join(path, filename)
    img = cv2.imread(img_path)
    img_encoded_str = base64.b64encode(cv2.imencode('.jpg', img)[1])

    # Here is just a toy example of labels.
    # The real labels can be generated from the annotation files
    # given by each dataset. The label is a list of dictionary
    # where each box with at least "rect" (xyxy mode) and "class"
    # fields. It can have any other fields given by the dataset.
    height = img.shape[0]
    width = img.shape[1]

    labels = []
    # labels.append({"rect": [1, 1, height, width], "class": ImageNet_classes[class_name]})
    labels.append({"rect": [1, 1, height, width], "class": class_name})

    row = [img_key, img_encoded_str]
    rows.append(row)

    row_label = [img_key, json.dumps(labels)]
    rows_label.append(row_label)

    row_hw = [img_key, json.dumps([{"height": height, "width": width}])]
    rows_hw.append(row_hw)

tsv_writer(rows, os.path.join(tsv_data_path, tsv_file))
tsv_writer(rows_label, os.path.join(tsv_data_path, label_file))
tsv_writer(rows_hw, os.path.join(tsv_data_path, hw_file))

# generate linelist file
generate_linelist_file(os.path.join(tsv_data_path, label_file), save_file=os.path.join(tsv_data_path, linelist_file))

yaml_dict = {"img": tsv_file,
            "label": label_file,
            "hw": hw_file,
            "linelist": linelist_file}

with open(op.join(tsv_data_path, '{}.test.yaml'.format(DATASET)), 'w') as file:
        yaml.dump(yaml_dict, file)

# To access a tsv file:
# 1) Use tsv_reader to read dataset in given order
# rows = tsv_reader("tools/mini_tsv/data/train.tsv")
# rows_label = tsv_reader("tools/mini_tsv/data/train.label.tsv")
# for row, row_label in zip(rows, rows_label):
#     img_key = row[0]
#     labels = json.loads(row_label[1])
#     img = img_from_base64(row[1])

# 2) use TSVFile to access dataset at any given row.
# tsv = TSVFile("tools/mini_tsv/data/train.tsv")
# row = tsv.seek(1) # to access the second row
# img_key = row[0]
# img = img_from_base64(row[1])

100%|█████████████████████████████████████████████████████████████████████████| 13000/13000 [03:02<00:00, 71.22it/s]
13000it [00:00, 113261.85it/s]


In [21]:
! ls {tsv_data_path}

ImageNet.test.hw.lineidx	ImageNet.test.linelist.tsv  train.labelmap.tsv
ImageNet.test.hw.tsv		ImageNet.test.tsv	    train.label.tsv
ImageNet.test.label.lineidx	ImageNet.test.yaml	    train.lineidx
ImageNet.test.label.tsv		train.hw.lineidx	    train.linelist.tsv
ImageNet.test.lineidx		train.hw.tsv		    train.tsv
ImageNet.test.linelist.lineidx	train.label.lineidx	    train.yaml


In [23]:
! cd scene_graph_benchmark && rm -rf output/{DATASET}
! cd scene_graph_benchmark && python -m torch.distributed.launch --nproc_per_node=8 tools/test_sg_net.py \
  --config-file sgg_configs/vgattr/vinvl_x152c4.yaml \
  TEST.IMS_PER_BATCH 8 \
  DATASETS.TEST "('{DATASET}.test.yaml', )"\
  MODEL.WEIGHT pretrained_model/vinvl_vg_x152c4.pth \
  MODEL.ROI_HEADS.NMS_FILTER 1 \
  MODEL.ROI_HEADS.SCORE_THRESH 0.2 \
  TEST.OUTPUT_FEATURE True \
  OUTPUT_DIR output/{DATASET} \
  DATA_DIR tools/mini_tsv/data \
  TEST.IGNORE_BOX_REGRESSION True \
  MODEL.ATTRIBUTE_ON True

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
['/nobackup-fast/ziyangc/scene_graph_benchmark/tools', '/u/z/i/ziyangc/anaconda3/envs/env1/lib/python38.zip', '/u/z/i/ziyangc/anaconda3/envs/env1/lib/python3.8', '/u/z/i/ziyangc/anaconda3/envs/env1/lib/python3.8/lib-dynload', '/u/z/i/ziyangc/.local/lib/python3.8/site-packages', '/u/z/i/ziyangc/anaconda3/envs/env1/lib/python3.8/site-packages', '/u/z/i/ziyangc/anaconda3/envs/env1/lib/python3.8/site-packages/apex-0.1-py3.8-linux-x86_64.egg', '/nobackup-fast/ziyangc/Oscar']
Traceback (most recent call last):
  File "tools/test_sg_net.py", line 8, in <module>
    from maskrcnn_benchmark.utils.env import setup_environment  # noqa F401 isort:skip
ModuleNotFoundError: No module named 'maskrcnn_benchmark'


In [5]:
import pandas as pd
import os
import json
import ast
import numpy as np
import base64

data_dir = 'scene_graph_benchmark/tools/mini_tsv/data'
sg_tsv = 'scene_graph_benchmark/output/{}/inference/vinvl_vg_x152c4/predictions.tsv'.format(DATASET)

# Height and width data
hw_df = pd.read_csv(os.path.join(data_dir, '{}.test.hw.tsv'.format(DATASET)), sep='\t', header=None, converters={1:ast.literal_eval}, index_col=0)
print(hw_df.head())

# bboxes, class, confidence and spacial features
df = pd.read_csv(sg_tsv, sep='\t', header=None, converters={1:json.loads})
df[1] = df[1].apply(lambda x:x['objects'])
print(df.head())

def generate_additional_features(rect, h, w):
    mask = np.array([w, h, w, h], dtype=np.float32)
    rect = np.clip(rect / mask, 0, 1)
    res = np.hstack((rect, (rect[3] - rect[1], rect[2] - rect[0])))
    return res.astype(np.float32)

def generate_features(x):
    idx, data, num_boxes = x[0], x[1], len(x[1] )
    h, w, features_arr = hw_df.loc[idx, 1][0]['height'], hw_df.loc[idx,1][0]['width'], []

    for i in range(num_boxes):
        features = np.frombuffer(base64.b64decode(data[i]['feature']),np.float32)
        pos_feat = generate_additional_features(data[i]['rect'],h,w)
        x = np.hstack((features,pos_feat))
        features_arr.append(x.astype(np.float32))

    features = np.vstack(tuple(features_arr))
    features = base64.b64encode(features).decode("utf-8")
    return {"features":features, "num_boxes":num_boxes}

def generate_labels(x):
    data = x[1]
    res = [{"class":el['class'].capitalize(),"conf":el['conf'], "rect": el['rect']} for el in data]
    return res

df['feature'] = df.apply(generate_features,axis=1)
df['feature'] = df['feature'].apply(json.dumps)

df['label'] = df.apply(generate_labels,axis=1)
df['label'] = df['label'].apply(json.dumps)

print(df.head())

output_dir = 'Oscar/oscar/my_data'
label_file = os.path.join(output_dir, '{}.label.tsv'.format(DATASET))
feature_file = os.path.join(output_dir, '{}.features.tsv'.format(DATASET))

from maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer
tsv_writer(df[[0,'label']].values.tolist(), label_file)
tsv_writer(df[[0,'feature']].values.tolist(), feature_file)

import yaml
import os.path as op
yaml_dict = {"label": '{}.label.tsv'.format(DATASET),
             "feature": '{}.features.tsv'.format(DATASET)}

with open(op.join(output_dir, '{}.test.yaml'.format(DATASET)), 'w') as file:
        yaml.dump(yaml_dict, file)

                                                        1
0                                                        
n02123597_n02123597_8644  [{'height': 398, 'width': 500}]
n02123597_n02123597_8723  [{'height': 500, 'width': 375}]
n02123597_n02123597_5224  [{'height': 375, 'width': 500}]
n02123597_n02123597_6464  [{'height': 375, 'width': 500}]
n02123597_n02123597_9585  [{'height': 375, 'width': 500}]


FileNotFoundError: [Errno 2] No such file or directory: 'scene_graph_benchmark/output/ImageNet/inference/vinvl_vg_x152c4/predictions.tsv'

In [ ]:
! cd Oscar && python -m torch.distributed.launch --nproc_per_node=8 build/lib/oscar/run_captioning.py \
    --eval_model_dir pretrained_models/image_captioning/coco_captioning_large_scst/checkpoint-4-50000/\
    --do_test \
    --test_yaml oscar/my_data/{DATASET}.test.yaml \
    --do_lower_case \
    --add_od_labels \
    --output_dir output/

In [ ]:
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import json
import numpy as np

base_inf_path = 'pretrained_models/image_captioning/pretrained_base/checkpoint-2000000/pred.coco_caption.test.beam1.max20.odlabels.tsv'
large_inf_path = 'pretrained_models/image_captioning/pretrained_large/checkpoint-1410000/pred.coco_caption.test.beam1.max20.odlabels.tsv'
coco_inf_path = 'Oscar/pretrained_models/image_captioning/coco_captioning_large_scst/checkpoint-4-50000/pred.coco_caption.{}.test.beam1.max20.odlabels.tsv'.format(DATASET)

df = pd.read_csv(coco_inf_path, sep='\t')
df.columns = ['img', 'raw_data']
objects = df.iloc[:,1].apply(json.loads)
df['caption'] = objects.apply(lambda x: x[0]['caption'])
df['conf'] = objects.apply(lambda x: x[0]['conf'])
df['class'] = df['img'].apply(lambda x: x.split('_')[0])
df['img'] = df['img'].apply(lambda x: '_'.join(x.split('_')[1:]))
print(df.head())

img_dir = data_path

In [ ]:
fig = plt.figure(dpi = 100, figsize=[20,20])

N = 6
for c, i in enumerate(np.random.randint(0, len(df), size=N)):
    test_image = df.iloc[i]['img']+'.jpg'
    test_class = df.iloc[i]['class']
    if DATASET in ['iNaturalist', 'Places']: test_class = 'images'
    test_caption = df.iloc[i]['caption']

    try:
        img = Image.open(os.path.join(img_dir, test_class, test_image))
    except:
        img = Image.open(os.path.join(img_dir, test_class, test_image))
    
    ax = fig.add_subplot(int(N/2), 2, c + 1)
    ax.imshow(img)
    ax.axis('off')
    txt = ax.text(0, -10, test_caption, wrap=True, fontsize='xx-large')
    txt._get_wrap_line_width = lambda : 800
plt.tight_layout()

In [ ]:
df[['img', 'caption', 'class']].to_csv('{}_captions.tsv'.format(DATASET), header=None, index=False, sep='\t')
! ls

In [ ]:
temp_df = pd.read_csv('imagenet_captions.tsv', sep='\t')